In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [3]:
# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))


29084464/29084464 [==============================] - 237s 8us/step


In [4]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Add custom classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Use 'sigmoid' for binary classification


In [6]:
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
    metrics=['accuracy', 'Precision', 'Recall', 'AUC']
)

In [9]:
batch = 32
classes = 'binary'

train='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/train'
test='C:/Users/Shusmita\Desktop\Oral Cancer/Augmented_Oral_Cancer/test'
val='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/Validate'

In [10]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validation_datagen= image.ImageDataGenerator()

test_datagen= image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)


validation_generator = validation_datagen.flow_from_directory(
    val,
    target_size = (224,224),
    batch_size = batch,
    shuffle=True,
    class_mode = classes)

test_generator = test_datagen.flow_from_directory(
    test,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)
class_names=validation_generator.class_indices
class_names
len(class_names)

Found 619 images belonging to 2 classes.
Found 130 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


2

In [11]:
# Train the model
epochs = 5  # You can adjust the number of epochs
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/5
20/20 [==============================] - 79s 4s/step - loss: 1.9066 - accuracy: 0.6026 - precision: 0.6038 - recall: 0.6077 - auc: 0.6061 - val_loss: 0.9809 - val_accuracy: 0.6385 - val_precision: 0.9048 - val_recall: 0.2969 - val_auc: 0.7111
Epoch 2/5
20/20 [==============================] - 83s 4s/step - loss: 1.1303 - accuracy: 0.6478 - precision: 0.6440 - recall: 0.6688 - auc: 0.6940 - val_loss: 0.6835 - val_accuracy: 0.7077 - val_precision: 0.9643 - val_recall: 0.4219 - val_auc: 0.8211
Epoch 3/5
20/20 [==============================] - 89s 4s/step - loss: 0.6551 - accuracy: 0.7351 - precision: 0.7458 - recall: 0.7170 - auc: 0.8001 - val_loss: 0.4503 - val_accuracy: 0.8000 - val_precision: 0.9130 - val_recall: 0.6562 - val_auc: 0.8844
Epoch 4/5
20/20 [==============================] - 120s 6s/step - loss: 0.5528 - accuracy: 0.7480 - precision: 0.7558 - recall: 0.7363 - auc: 0.8223 - val_loss: 0.3810 - val_accuracy: 0.8308 - val_precision: 0.8750 - val_recall: 0.7656 - val

In [12]:
score=model.evaluate(validation_generator)
score

5/5 [==============================] - 13s 2s/step - loss: 0.5572 - accuracy: 0.7000 - precision: 0.9310 - recall: 0.4219 - auc: 0.8925


[0.5571834444999695,
 0.699999988079071,
 0.931034505367279,
 0.421875,
 0.8925188779830933]

In [13]:
score=model.evaluate(test_generator)
score

5/5 [==============================] - 14s 3s/step - loss: 0.5696 - accuracy: 0.7143 - precision: 0.9250 - recall: 0.5000 - auc: 0.8681


[0.569620668888092,
 0.7142857313156128,
 0.925000011920929,
 0.5,
 0.8681408166885376]

In [14]:
score=model.evaluate(train_generator)
score

20/20 [==============================] - 102s 5s/step - loss: 0.4518 - accuracy: 0.7787 - precision: 0.9579 - recall: 0.5852 - auc: 0.9289


[0.45179083943367004,
 0.7786752581596375,
 0.9578947424888611,
 0.5852090120315552,
 0.9289002418518066]